# Notebook to test plotting of UAH MRR data

In [ ]:
import numpy as np
import numpy.ma as ma
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.ticker as ticker
import matplotlib.dates as dates
from mpl_toolkits.axes_grid1 import ImageGrid,make_axes_locatable,host_subplot
#from mpl_toolkits.basemap import Basemap
from datetime import datetime, timedelta
import sys
import os
import pandas as pd
import xarray as xr
import pyPIPS.utils as utils
import pyPIPS.thermolib as thermo
import pyPIPS.DSDlib as dsd
#import pyPIPS.disdrometer_module as dis
import pyPIPS.plotmodule as PIPSplot
#import pyPIPS.simulator as sim
import pyPIPS.pips_io as pipsio
import pyPIPS.PIPS as pips
import pyPIPS.parsivel_params as pp
import pyPIPS.parsivel_qc as pqc
import pyPIPS.polarimetric as dualpol
import pyPIPS.timemodule as ptime
# from pyCRMtools.modules import plotmodule as plotmod
from pyCRMtools.modules import utils as CRMutils
# from pyCRMtools.pycaps import arps_read
# from pyCRMtools.pycaps import pycaps_fields
# from pyCRMtools.pycaps import calvars_radar as radar
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
MRR_dir = '/Users/dawson29/Projects/PERiLS/obsdata/2022/UAH_MRR'
MRR_file1 = 'RaDAPS_MRR_20220330.nc'
MRR_file2 = 'RaDAPS_MRR_20220331.nc'

MRR_path1 = os.path.join(MRR_dir, MRR_file1)
MRR_path2 = os.path.join(MRR_dir, MRR_file2)

In [ ]:
MRR_ds1 = xr.load_dataset(MRR_path1)
MRR_ds2 = xr.load_dataset(MRR_path2)
# MRR_ds = xr.merge(MRR_ds1, MRR_ds2)
# MRR_ds

In [ ]:
MRR_ds1

In [ ]:
MRR_ds2

In [ ]:
MRR_ds = xr.concat([MRR_ds1, MRR_ds2], dim='time')
starttime = '2022-03-30T22:49'
endtime = '2022-03-31T01:34'
MRR_ds = MRR_ds.sel(time=slice(starttime, endtime))

In [ ]:
MRR_ds

In [ ]:
MRR_ds = MRR_ds.set_coords('MRR_D')
# MRR_ds = MRR_ds.where(MRR_ds['MRR_D'] >= 0., drop=True)
# MRR_ds = MRR_ds.where(np.isfinite(MRR_ds['MRR_D']), drop=True)
# MRR_ds = MRR_ds.dropna(dim='MRR spectralclass', subset=['MRR_D'])

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
MRR_ds['MRR_Capital_Z'].plot(ax=ax, x='time')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
MRR_ds['MRR_RR'].plot(ax=ax, x='time', robust=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))

# I suspect the units are actually in m^-4, not m^-3 mm^-1, so divide by 1000 here to fix that
log_N = np.log10(MRR_ds['MRR_N'] / 1000.)
log_N = log_N.where(log_N > -1.0)


log_N_plot = log_N.isel({'MRR rangegate': 2})
# log_N.isel({'MRR rangegate': 2}).plot(x='time', y='MRR_D', vmin=-1.0, vmax=3.0, cmap='viridis')

# print(log_N_plot.sizes)

print(log_N_plot['time'].shape, log_N_plot['MRR_D'].shape)
# Need to promote the time dimension to 2D with same shape as MRR_D, which is dimensioned by (time, MRR spectralclass). pcolor requires
# that both x and y coordinates are the same shape...
time_plot = np.tile(log_N_plot['time'], (log_N_plot.sizes['MRR spectralclass'], 1)).T
print(time_plot.shape)

# Have to use pcolor here instead of pcolormesh because MRR_D contains nans
ax.pcolor(time_plot, log_N_plot['MRR_D'], log_N_plot.values, vmin=-1.0, vmax=3.0, cmap='viridis')
# ax.set_ylim(0., 9.)

In [ ]:
MRR_D = MRR_ds['MRR_D']
MRR_D.isel({'MRR rangegate': 20}).plot(x='time')

In [ ]:
# Function to convert combined time string to hours, minutes, seconds
def parse_time(time_str):
    hour = int(time_str[:2])
    minute = int(time_str[2:4])
    second = int(time_str[4:])
    return hour, minute, second

# Function to convert year, Julian day, and time to datetime
def julian_to_datetime(row):
    time_int = int(row['HHMMSS'])  # Annoying that this is read in by default as a float, so have to do some finagling
    hour, minute, second = parse_time(f'{time_int:06d}')
    return pd.to_datetime(f"{int(row['year'])}-01-01") + pd.to_timedelta(row['julian_day'] - 1, unit='d') + \
           pd.to_timedelta(hour, unit='h') + pd.to_timedelta(minute, unit='m') + pd.to_timedelta(second, unit='s')

def read_UAH_parsivel_to_xarray(parsivel_path):
    # Construct the header, because the files don't have one
    header1 = ['year', 'julian_day', 'HHMMSS', 'sensor_temp', 'pcount', 'RR']
    header2 = [f'N({avg_diameter:.4f})' for avg_diameter in pp.avg_diameter]
    header3 = [f'V({avg_fallspeed:.4f})' for avg_fallspeed in pp.avg_fallspeed]
    header4 = ['Unknown']  # I think this is the reflectivity, but the documentation doesn't say
    header = header1 + header2 + header3 + header4
    parsivel_df = pd.read_csv(parsivel_path, header=None, names=header, index_col=False)
    # Apply the function to create a new datetime column
    parsivel_df['time'] = parsivel_df.apply(julian_to_datetime, axis=1)
    # Set the new datetime column as the index
    parsivel_df.set_index('time', inplace=True)
    # Drop the existing "year", "julian_day", and "time" columns
    parsivel_df.drop(columns=['year', 'julian_day', 'HHMMSS'], inplace=True)

    # Convert to xarray Dataset
    parsivel_ds = parsivel_df.to_xarray()

    # We're not done yet. We want the N(D) and V(D) to be two new DataArrays in the new DataSet, so we have to
    # wrangle the existing variables into submission by concatenating them along new diameter and velocity dimensions

    # Start with ND
    NDs = [parsivel_ds[var].expand_dims(dim='diameter', axis=0) for var in header2]

    # Assign the new dimension coordinate values
    for i, da in enumerate(NDs):
        NDs[i] = da.assign_coords(diameter=[pp.avg_diameter[i]])
    
    # Concatenate the DataArrays along the new dimension
    ND = xr.concat(NDs, dim='diameter')
    
    # Drop the original variables from the dataset
    parsivel_ds = parsivel_ds.drop_vars(header2)
    
    # Add the concatenated variable back to the dataset
    parsivel_ds['ND'] = ND

    # Now do the same for VD
    VDs = [parsivel_ds[var].expand_dims(dim='fallspeed', axis=0) for var in header3]

    # Assign the new dimension coordinate values
    for i, da in enumerate(VDs):
        VDs[i] = da.assign_coords(fallspeed=[pp.avg_fallspeed[i]])
    
    # Concatenate the DataArrays along the new dimension
    VD = xr.concat(VDs, dim='fallspeed')
    
    # Drop the original variables from the dataset
    parsivel_ds = parsivel_ds.drop_vars(header3)
    
    # Add the concatenated variable back to the dataset
    parsivel_ds['VD'] = VD
    
    return parsivel_ds

parsivel_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/PERiLS/obsdata/2022/UAH_MAPNET_Parsivel'
parsivel_file1 = 'radaps_20220330_parsivel.dat'
parsivel_file2 = 'radaps_20220331_parsivel.dat'

parsivel_path1 = os.path.join(parsivel_dir, parsivel_file1)
parsivel_path2 = os.path.join(parsivel_dir, parsivel_file2)

parsivel_ds1 = read_UAH_parsivel_to_xarray(parsivel_path1)
parsivel_ds2 = read_UAH_parsivel_to_xarray(parsivel_path2)

parsivel_ds = xr.concat([parsivel_ds1, parsivel_ds2], dim='time')
parsivel_ds


# def read_UAH_parsivel_to_xarray(parsivel_path):
#     """Given a UAH Parsivel file path, read it into an xarray Dataset"""

#     parsivel_df = pd.read_csv(parsivel_path)
#     # Set the time index

    
#     probe_filename = f"0{probe_id}_{deployment_name}_level3.txt"
#     probe_filepath = os.path.join(dir, probe_filename)
#     probe_df = pd.read_csv(probe_filepath)
#     # Set the time index
#     probe_df['Time'] = pd.DatetimeIndex(probe_df['Time'])
#     probe_df.set_index("Time", inplace=True)
#     probe_ds = probe_df.to_xarray()

#     # Set location, probe id, start and end times, etc., as attributes
#     probe_ds.attrs['probe_name'] = probe_id
#     lat = loc_ds.sel(ID=probe_id)['Latitude'].to_numpy().item()
#     lon = loc_ds.sel(ID=probe_id)['Longitude'].to_numpy().item()
#     elev = loc_ds.sel(ID=probe_id)['Elevation'].to_numpy().item()
#     location_tuple = (lat, lon, elev)
#     probe_ds.attrs['location'] = str(location_tuple)
#     probe_ds.attrs['Array_Type'] = "Fine"
#     starting_time = probe_ds['Time'][0].dt.strftime("%Y%m%d%H%M%S").item()
#     ending_time = probe_ds['Time'][-1].dt.strftime("%Y%m%d%H%M%S").item()
#     probe_ds.attrs['starting_time'] = starting_time
#     probe_ds.attrs['ending_time'] = ending_time

#     return probe_ds

In [ ]:
parsivel_ds = parsivel_ds.sel(time=slice(starttime, endtime))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
parsivel_ds['ND'].plot(ax=ax, vmin=-1, vmax=3, cmap='viridis')
ax.set_ylim(0., 9.)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
parsivel_ds['Unknown'].plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
MRR_ds['MRR_Capital_Z'].isel({'MRR rangegate': 0}).plot(ax=ax)

In [ ]:
# Plot observed DSD meteograms

ND = parsivel_ds['ND'].T
ZH = parsivel_ds['Unknown']
ZH_MRR = MRR_ds['MRR_Capital_Z'].isel({'MRR rangegate': 1})
radmidtimes = ZH_MRR['time']
# print(ZH_MRR)

# Truncate diameter range to less than 9 mm
D_max = 9.
D_range_full = ND['diameter'].values
D_max_ind = np.searchsorted(D_range_full, D_max)
D_range = D_range_full[:D_max_ind]
print(D_max_ind, D_range)
ND_trunc = ND.isel(diameter=slice(0, D_max_ind))

# Get PSD times
PSD_datetimes = pips.get_PSD_datetimes(ND)
PSD_datetimes_dict = pips.get_PSD_time_bins(PSD_datetimes)
PSDstarttimes = dates.date2num(PSD_datetimes_dict['PSD_datetimes_edges'])
PSDmidtimes = dates.date2num(PSD_datetimes_dict['PSD_datetimes_centers'])

# Prepare axis parameters
# We'll use the model times for the boundaries of the x-axis
timelimits = [PSDstarttimes[0], PSDstarttimes[-1]]
diamlimits = [0.0, 9.0]
diamytick = 1.0
DSDtype = 'observed'
locator = dates.MinuteLocator(byminute=[0,15,30,45])
minorlocator = dates.MinuteLocator(byminute=range(0,60,5))
dateformat = '%H:%M'
formatter = dates.DateFormatter(dateformat)

axparams = {'majorxlocator': locator, 'majorxformatter': formatter,
            'minorxlocator': minorlocator,
            'axeslimits': [timelimits, diamlimits],
            'majorylocator': ticker.MultipleLocator(base=diamytick),
            'axeslabels': [None, 'D (mm)']}

# It seems that this is already in log units for the UAH Parsivel
logND = ND_trunc # np.log10(ND_trunc)
logND = logND.where(logND > -1.0)

# Ok, now we should have everything ready to go to plot the meteograms.
# Let'er rip!

# D0 = D0_PIPS_interp_to_model_times_dict[dis_name] * 1000. # Get to mm again
# dBZ = dBZ_PIPS_interp_to_model_times_dict[dis_name]
# ZDR = dualpol_PIPS_interp_to_model_times_dict[dis_name]['ZDR']

diameter_bin_edges = pp.parsivel_parameters['diameter_bin_edges_mm']
diameter_bin_edges = diameter_bin_edges[:D_max_ind+1]

disvars = {'diameter_bin_edges': diameter_bin_edges, 'PSDstarttimes': PSDstarttimes,
           'PSDmidtimes': PSDmidtimes, 'logND': logND.T, 'REF': ZH} # , 'D_m': Dm_sorted_PIPS_da, 'dBZ': ZH_sorted_PIPS_da, 
           # 'ZDR': ZDR_sorted_PIPS_da}

radvars = {'radmidtimes': radmidtimes, 'REF': ZH_MRR}

plot_dir = os.path.join(parsivel_dir, 'plots')
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
dis_plot_name = 'UAH_' + DSDtype
PIPSplot.plotDSDmeteograms(dis_plot_name, plot_dir, axparams, disvars, radvars=radvars, close_fig=False)